In [15]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import random
from tqdm import tqdm

embeddings = torch.load(
    "/home/prz/PROJECTS/rvae_reproduction/results/embeddings/fmnist_embeddings.pt"
)

/tmp/ipykernel_81288/2388215574.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load(


---

In [16]:
class SimpleClassifier(nn.Module):

    def __init__(self, in_features: int, hidden_dim: int, out_features: int) -> None:
        super().__init__()
        self.ff = nn.Sequential(
            nn.Linear(in_features=in_features, out_features=hidden_dim),
            nn.ReLU(),
            nn.Linear(in_features=hidden_dim, out_features=out_features),
        )

    def forward(self, x):
        return self.ff(x)


# simple training loop function
def train_model(model, dataloader, criterion, optimizer, num_epochs=10, seed=None):
    if seed is not None:
        torch.manual_seed(seed)
        random.seed(seed)
        np.random.seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    device = next(model.parameters()).device
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0.0
        for inputs, targets in tqdm(dataloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * inputs.size(0)
        epoch_loss = total_loss / len(dataloader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")


def predict_model_argmax(model, dataloader):
    device = next(model.parameters()).device
    model.eval()
    all_preds = []
    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            preds = model(inputs).argmax()
            all_preds.append(preds)
    return torch.stack(all_preds, dim=0)

---

In [ ]:
def train_and_predict_classifiers(
    embeddings: dict, method_type: str, seed: int, n_models: int = 5
):
    device = torch.device("cuda") if torch.cuda.is_available else torch.device("cpu")
    print(f"Using device: {device}")

    embeddings[method_type][seed]
    tensor_dataset_train = TensorDataset(
        torch.tensor(embeddings[method_type][seed]["train"]["mus"]),
        torch.tensor(embeddings[method_type][seed]["train"]["labels"]),
    )
    tensor_dataset_test = TensorDataset(
        torch.tensor(embeddings[method_type][seed]["test"]["mus"]),
        torch.tensor(embeddings[method_type][seed]["test"]["labels"]),
    )
    models = [
        SimpleClassifier(
            embeddings[method_type][seed]["train"]["mus"].shape[-1], 20, 10
        ).to(device)
        for _ in range(n_models)
    ]
    for model in models:
        train_model(
            model,
            dataloader=DataLoader(tensor_dataset_train),
            criterion=nn.CrossEntropyLoss(),
            optimizer=optim.AdamW(model.parameters(), lr=1e-4),
        )
    predictions = [
        predict_model_argmax(model, DataLoader(tensor_dataset_test)) for model in models
    ]
    return predictions

In [18]:
predictions = {}

for model_type in embeddings.keys():
    predictions[model_type] = {}
    for seed in embeddings[model_type].keys():
        predictions[model_type][seed] = train_and_predict_classifiers(
            embeddings=embeddings, method_type="RVAE", seed=5
        )

/tmp/ipykernel_81288/2722621096.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(embeddings[method_type][seed]["train"]["mus"]),
/tmp/ipykernel_81288/2722621096.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(embeddings[method_type][seed]["train"]["labels"]),
/tmp/ipykernel_81288/2722621096.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(embeddings[method_type][seed]["test"]["mus"]),
/tmp/ipykernel_81288/2722621096.py:11: UserWarning: To copy construct from a tensor, it is recommended to 

KeyboardInterrupt: 

---